# Tuning the Top-k Query parameter

First we set up the environment

In [1]:
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd
from time import time
import torch
from Enrichers import Enricher, get_enricher, ENRICHMENT_FN
from database.database import DatabaseProcessor
from Embedders import Embedder, get_embedder
from tqdm import tqdm

load_dotenv('.env', override=True)

# Database setup
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db = DatabaseProcessor(db_params)
db.test_connection()

device = 'cuda' if torch.cuda.is_available(
) else 'mps' if torch.mps.is_available() else 'cpu'
print(f"Using device: {device}")

=================================CONFIG=================================
Database         User             Host                             Port            
citeline_db      bbasseri         localhost                        5432            
Database version: ('PostgreSQL 17.3 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Using device: mps


## Investigating precision over `k`

For our various embedding models and enrichment strategies, we want to know the smallest `top_k` value that will still retrieve the target reference for a given sentence. 

To investigate this, we'll sample 100 examples from the non-trivial training data. Each example typically has 1-2 target DOI's. For each example, we'll query the database with a large `top_k` parameter to start, so we can be sure the database returns the target references. Then we can ask at what index in the query results does a target DOI first appear. Ideally, the ranks will all be very high, indicated by having *low* indices in the query results. We also expect enriched examples to have their target doi's higher ranked (lower indices).

In [ ]:
from database.database import SingleQueryResult

data = pd.read_json('data/dataset/split/train.jsonl', lines=True)
examples = data.sample(100, random_state=42)

def lowest_index_matching_doi(target_doi: str, query_results: list[SingleQueryResult]) -> int:
    """
    Returns the first index of the query results where the chunk doi matches the target doi.
    If no match is found, returns -1.
    """
    for i, result in enumerate(query_results):
        if target_doi == result.doi:
            return i
    return -1


def get_ranks(
    example: pd.Series,
    embedder: Embedder,
    enricher: Enricher,
    table_name: str,
    top_k: int,
    ef_search: int,
    metric: str = 'vector_cosine_ops'
) -> list[int]:
    target_dois = example['citation_dois']

    # Prepare query vector
    enriched_sentence = enricher.enrich(example=example)
    embedding = embedder([enriched_sentence])[0]

    # Query
    start = time()
    query_results = db.query_vector_table(
        table_name=table_name,
        query_vector=embedding,
        metric=metric,
        use_index=True,
        top_k=top_k,
        ef_search=ef_search
    )
    
    ranks = [lowest_index_matching_doi(
        target_doi=doi, query_results=query_results) for doi in target_dois]
    return ranks

In [ ]:
from itertools import product

embedding_models = ['BAAI/bge-small-en']
if device == 'cuda':
    embedding_models += ['bert-base-uncased', 'adsabs/astroBERT']
enrichers = ENRICHMENT_FN.keys()
combos = list(product(embedding_models, enrichers))

model_names_to_tables = {
    'BAAI/bge-small-en': 'bge',
    'bert-base-uncased': 'bert_hnsw',
    'adsabs/astroBERT': 'astrobert_hnsw'
}

# Dict mapping "model name": pd.DataFrame() of results (enrichment functions used -> columns)
results = {model: pd.DataFrame() for model in embedding_models}

for embedding_model, enricher_name in combos:
    print(f"Embedding model: {embedding_model}, Enricher: {enricher_name}")
    embedder = get_embedder(embedding_model, device=device)
    enricher = get_enricher(enricher_name)
    table_name = model_names_to_tables[embedding_model]
    ranks = []
    for i in tqdm(range(len(examples))):
        example = examples.iloc[i]
        ranks += get_ranks(
            example=example, 
            embedder=embedder, 
            enricher=enricher,
            table_name=table_name,
            top_k=350000,
            ef_search=350000)
    series = pd.Series(ranks)
    results[embedding_model][enricher_name] = series



Embedding model: BAAI/bge-small-en, Enricher: identity


  2%|▏         | 2/100 [00:00<00:15,  6.52it/s]

  Query execution time: 0.02 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.02 seconds
  Query fetch time: 0.00 seconds


  4%|▍         | 4/100 [00:00<00:11,  8.43it/s]

  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.02 seconds
  Query fetch time: 0.00 seconds


...

  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds


 97%|█████████▋| 97/100 [00:06<00:00, 16.31it/s]

  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds


100%|██████████| 100/100 [00:06<00:00, 15.81it/s]

  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds
  Query execution time: 0.01 seconds
  Query fetch time: 0.00 seconds


In [6]:
results['BAAI/bge-small-en']

,identity,add_abstract,add_title,add_title_and_abstract,add_previous_3_sentences,add_previous_7_sentences,add_headers_and_previous_3_sentences,add_headers_and_previous_7_sentences
0,-1,-1,4,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,3,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
107,0,-1,-1,-1,-1,-1,-1,-1
108,-1,-1,-1,-1,-1,-1,-1,-1
109,-1,-1,-1,-1,4,4,-1,-1
110,-1,-1,-1,-1,-1,-1,-1,-1


In [8]:
model_names_to_filenames = {
    'BAAI/bge-small-en': 'bge',
    'bert-base-uncased': 'bert',
    'adsabs/astroBERT': 'astrobert'
}

# Save the results to CSV files
for model_name, df in results.items():
    filename = f"tests/{model_names_to_filenames[model_name]}_ranks.csv"
    df.to_csv(filename, index=False)
    print(f"Saved results for {model_name} to {filename}")

Saved results for BAAI/bge-small-en to tests/bge_ranks.csv
